In [1]:
import os
import json
import random

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.util.tools_fabric import ToolsFabric

from experiments.models.experiment_models import LabeledQAPairFacts
from experiments.util.fact_score.fact_scorer import FactScorer

In [2]:
USE_MCP = False
DEBUG_MODE = False
INCLUDE_WEBSEARCH_TOOL = True

fact_scorer = FactScorer()
path_to_ressources = "./resources/01_facts/"

files = {
    "extensibility": "extensibility_assistance_facts.json",
    "malaysia": "malaysia_support_facts.json",
    "peppol": "peppol_support_facts.json",
    "all": "all_cases_facts.json",
}

In [3]:
file_path = os.path.abspath(path_to_ressources + files["all"])


with open(file_path, encoding="utf8") as f:
    data = json.load(f)
    qa_data_set = [LabeledQAPairFacts(**item) for item in data]

In [4]:
# Select 3 random elements
random_qa_pairs = random.sample(qa_data_set, 3)

In [5]:
triage_agent = Triage()

for qa_pair in random_qa_pairs:
    print(f"Asking agent question with id: {qa_pair.id}")
    query_categorization = triage_agent.triage_user_message(
        user_message=qa_pair.question
    )
    if DEBUG_MODE:
        print(f"\tQuery categorization: {query_categorization}")

    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=query_categorization["category"],
        include_web_search=INCLUDE_WEBSEARCH_TOOL,
    )

    if USE_MCP:
        async with MultiServerMCPClient(tools) as client:
            agent = ReActAgent(tool_list=client.get_tools())
            if DEBUG_MODE:
                print(f"\t{agent.create_sys_prompt()}\n")
            await agent.arun_agent_with_input(
                user_message=query_categorization["user_query"], debug=DEBUG_MODE
            )
    else:
        agent = ReActAgent(tool_list=tools)
        if DEBUG_MODE:
            print(f"\t{agent.create_sys_prompt()}\n")
        agent.run_agent_with_input(
            user_message=query_categorization["user_query"], debug=DEBUG_MODE
        )

    run_data = agent.get_execution_data()

    if DEBUG_MODE:
        run_data.pretty_print()

    print("\tFinished agent execution")

    print("\tStarted FactScore calculation")
    output = await fact_scorer.get_fact_score(
        facts=qa_pair.facts,
        knowledge_source=run_data.final_output,
        debug=DEBUG_MODE,
    )
    print(f"\tFact score: {output}")

Asking agent question with id: EA-32
	Finished agent execution
	Started FactScore calculation
	Fact score: 0.0
Asking agent question with id: PS-3
	Finished agent execution
	Started FactScore calculation
	Fact score: 0.0
Asking agent question with id: EA-25
	Finished agent execution
	Started FactScore calculation
	Fact score: 0.0
